In [25]:
import src.config as config

from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h')
print(f'{current_date=}')

fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2024-01-28 04:00:00')


In [26]:
print(f'{fetch_data_to}')
print(f'{fetch_data_from} 28 days ago')

2024-01-28 04:00:00
2023-12-31 04:00:00 28 days ago


In [27]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:

    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')
    
    rides=load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [28]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-12-31 04:00:00'), to_date_=Timestamp('2023-01-29 04:00:00')
File 2023-01 was already in local storage


File 2023-01 was already in local storage


In [29]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
100%|██████████| 257/257 [00:00<00:00, 278.32it/s]


In [30]:
ts_data.dtypes

pickup_hour           datetime64[ns]
rides                          int64
pickup_location_id             int64
dtype: object

In [31]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].view('int64') // 10**6

In [41]:
ts_data.dtypes

pickup_hour           datetime64[ns, UTC]
rides                               int64
pickup_location_id                  int64
pickup_ts                           int64
dtype: object

In [42]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()

feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Times-series data at hourly frequency",
    primary_key = ['pickup_location_id','pickup_ts'],
    event_time = 'pickup_ts',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.


In [43]:
feature_group.insert(ts_data, write_options={"wait_for_jobs": False})

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - pickup_ts (type: 'bigint') does not exist in feature group.